<a href="https://colab.research.google.com/github/seosztt/Deep_Learning_from_Scratch_2/blob/master/chapter4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
!ls

sample_data


In [3]:
cd '/gdrive/MyDrive/Deep_Learning_from_Scratch/ch04'

/gdrive/MyDrive/Deep_Learning_from_Scratch/ch04


In [4]:
import sys
sys.path.append('..')
from common.np import *
from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        W_in = 0.01 * np,random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        
        
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:,i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

In [4]:
import sys
sys.path.append('..')
import numpy as np
from common import config
config.GPU = True
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
from datasets import ptb

window_size=5
hidden_size=100
batch_size = 100
max_epoch=10

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params={}
params['word_vecs'] = word_vecs.astype(np.floar16)
params['word_to_id'] = word_to_id
parpam['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

------------------------------------------------------------
                       GPU Mode (cupy)
------------------------------------------------------------

| 에폭 1 |  반복 1 / 9295 | 시간 3[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 3[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 4[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 4[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 4[s] | 손실 4.04
| 에폭 1 |  반복 101 / 9295 | 시간 5[s] | 손실 3.91
| 에폭 1 |  반복 121 / 9295 | 시간 5[s] | 손실 3.77
| 에폭 1 |  반복 141 / 9295 | 시간 6[s] | 손실 3.62
| 에폭 1 |  반복 161 / 9295 | 시간 6[s] | 손실 3.49
| 에폭 1 |  반복 181 / 9295 | 시간 7[s] | 손실 3.36
| 에폭 1 |  반복 201 / 9295 | 시간 7[s] | 손실 3.27
| 에폭 1 |  반복 221 / 9295 | 시간 7[s] | 손실 3.16
| 에폭 1 |  반복 241 / 9295 | 시간 8[s] | 손실 3.09
| 에폭 1 |  반복 261 / 9295 | 시간 8[s] | 손실 3.03
| 에폭 1 |  반복 281 / 9295 | 시간 9[s] | 손실 2.96
| 에폭 1 |  반복 301 / 9295 | 시간 9[s] | 손실 2.91
| 에폭 1 |  반복 321 / 9295 | 시간 9[s] | 손실 2.88
| 에폭 1 |  반복 341 / 9295 | 시간 10[s] | 손실 2.86
| 에폭 1 |  반복 361 / 9295 | 시간 10[s] | 손실 2.82
| 에폭 1

KeyboardInterrupt: ignored

In [ ]:
import sys
sys.path.append('..')
from common.util import most_similar
import pickle

pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id =params['word_to_id']
    id_to_word = params['id_to_word']

querys=['you','year','car','toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625


In [ ]:
from ch04.eval import analogy
analogy('king', 'man','queen', word_to_id, id_to_word, word_vecs)
analogy('take','took','go',word_to_id, id_to_word, word_vecs)
analogy('car','cars','child',word_to_id, id_to_word, word_vecs)
analogy('good','better','bad',word_to_id, id_to_word, word_vecs)


[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slower: 4.734375
 greater: 4.671875
